## 2. Creating an Expressive 3D Morphable Model

#### Prerequisites

- Access to a 3D facial shape model of identity
- Access to a 3D facial shape model of expression (a 'blendshape' model)

The authors of:

> Zhu, Xiangyu, et al. "Face alignment across large poses: A 3d solution." CVPR 2016

Provide data [on their website](http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/main.htm) which can be used to bootstrap the expression model, and the LSFM model can be used for identity variation. As an example, we show loading this data into the format we need to proceed, and then demonstrate building the combined expression/identity model. The file this example follows is:
```
300W-3D.zip
```
and also requires access to the following file (the LSFM model):
```
all_all_all.mat
```
place the two files together in a directory and unzip the zip file in place. Replace `DATA_PATH` with the parent folder path.

In [ ]:
from pathlib import Path
import numpy as np

import scipy.io as sio
import menpo.io as mio
import menpo3d.io as m3io

from menpo.shape import TriMesh, PointCloud
from menpo.model import PCAModel

# Replace DATA_PATH with the path to your data. It should have subdirectories:
#  300W-3D/
#  all_all_all.mat
DATA_PATH = Path('~/Dropbox/itwmm_src_data/').expanduser()

In [ ]:
def generate_id_exp_model(data_path):
    # Load the LSFM ID model
    lsfm_model = m3io.import_lsfm_model(data_path / 'all_all_all.mat')
    
    # The number of ID and expression deformation components
    n_id = lsfm_model.n_active_components
    n_exp = 28 
    
    # Multiply the LSFM components with a scale factor, and bake in the eigenvalues.
    # The scale factor is to normalize weightings between ID and expression models.
    scale = np.sqrt(lsfm_model.eigenvalues[:n_id])
    i_raw = lsfm_model.components[:n_id] * scale[:, None]
    
    # Load the expression deformations from the data, and again scale for balancing
    scale_exp_mean = 1.0006846894227463e-05
    scale_exp_comps = scale_exp_mean / 2.5
    e_raw = sio.loadmat(str(data_path / '300W-3D' / 'Code' / 'Model_Exp.mat'))['w_exp'].T[:-1] * scale_exp_comps 
    e_mean = sio.loadmat(str(data_path / '300W-3D' / 'Code' / 'Model_Exp.mat'))['mu_exp'].T * scale_exp_mean
    
    # We now just need to combine the two models.
    id_ind = np.arange(n_id)
    exp_ind = np.arange(n_id, n_id + n_exp)
    components = np.zeros([n_id + n_exp, i_raw.shape[1]])
    components[id_ind] = i_raw
    components[exp_ind] = e_raw
    mean_combined = TriMesh(lsfm_model.mean().points + e_mean.reshape((-1,3)), lsfm_model.mean().trilist) 
    
    # iBUG 68 landmarks 
    lms_vertex_indices = [
        21868, 22404, 22298, 22327, 43430, 45175, 46312, 47132, 47911,
        48692, 49737, 51376, 53136, 32516, 32616, 32205, 32701, 38910,
        39396, 39693, 39934, 40131, 40843, 41006, 41179, 41430, 13399,
        8161,  8172,  8179,  8185,  5622,  6881,  8202,  9403, 10764,
        1831,  3887,  5049,  6214,  4805,  3643,  9955, 11095, 12255,
        14197, 12397, 11366,  5779,  6024,  7014,  8215,  9294, 10267,
        10922,  9556,  8836,  8236,  7636,  6794,  5905,  7264,  8223,
        9063, 10404,  8828,  8228,  7509
    ]
    lms = PointCloud(mean_combined.points[lms_vertex_indices])

    # Initialize a Menpo PCA Model from the raw data.
    # Note that we fix the eigenvalues to 1, as we choose to rescale
    # the components to 'bake-in' the eigenvalues above.
    shape_model = PCAModel.init_from_components(components, 
                                                np.ones(components.shape[1]), 
                                                mean_combined, 
                                                lsfm_model.n_samples, 
                                                True)
    
    return shape_model, lms, id_ind, exp_ind

In [ ]:
# Blend the ID and expression models into one, and note
# The indices into the model for both indivual components.
shape_model, lms, id_ind, exp_ind = generate_id_exp_model(DATA_PATH)

In [ ]:
%matplotlib qt
# generate a random expression to see if it's sensible
n_c = shape_model.n_components

# The std. dev weighting to use
sigma = 1

weights = np.random.multivariate_normal(np.zeros(n_c), np.eye(n_c) * sigma, 1).ravel()

synthesized_vector = shape_model.mean_vector + np.sum(weights[:, None] * shape_model.components, axis=0)
synthesized_instance = shape_model.template_instance.from_vector(synthesized_vector)
synthesized_instance.view()

# Exporting the combined ID/expression shape model

To use this model we just need to save it out and load it in to the fitting notebooks. Note that we need to save out some metadata along with the raw PCA basis to be able to use this again in the future. In particular, we need:

- The record of which components are identitiy and which are expression (`id_ind` and `exp_ind`)
- The vertex indices of the sparse set of landmarks on the model

We also need the ITW texture model we use to be in perfect correspondence with the shape model provided here, and for the landmarks in data we fit to be of the same configuration.

In [ ]:
mio.export_pickle(
    {
        'shape_model': shape_model,
        'id_ind': id_ind,
        'exp_ind': exp_ind,
        'lms': lms
    }, 
    DATA_PATH / 'id_exp_shape_model.pkl',
    protocol=4, overwrite=True)